In [2]:

import sys 
sys.path.append(r'C:\Users\sergiojesus\Desktop\Diogo2017\EQS_LOCAL\Bitbucket_eqs\analytics_stat')
import distributions as di

import warnings
warnings.filterwarnings('ignore')

import datetime
import scipy.stats as s
#import scipy.optimize as opt
#import scipy.special as special
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib inline
# %matplotlib notebook

import os
import json
import pandas as pd
import qgrid
from IPython.display import HTML, Image, display, clear_output
# display(HTML("<style>.container { width:95% !important; }</style>"))

# py.sign_in('diogo.antunes.goncalves', 'kt7T5JadNn2AhWk3DdPR')

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import traitlets
from tkinter import Tk, filedialog

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from spacy.tokenizer import Tokenizer
from spacy.vocab import Vocab
from spacy.language import Language
import spacy
import re
import unidecode
import edit_distance_nltk

nlp = Language(Vocab()) #Spacy variables
tokenizer = Tokenizer(nlp.vocab)
def tokenization_process(string):
    """tokenizes one string, removing not used punctuation in portuguese language in the process"""
    aux = tokenizer(string)
    output = list()
    for index, word in enumerate(aux):
        if re.search('([A-Za-zÀ-ÿ]+(-|)[A-Za-zÀ-ÿ]+|[A-Za-zÀ-ÿ]+)', str(word)):
                output.append(str(re.search('([A-Za-zÀ-ÿ]+(-|)[A-Za-zÀ-ÿ]+|[A-Za-zÀ-ÿ]+)', str(word)).group(0)).lower())
    return output

def toptokens(Tfidfmatrix, tokens, n=20):
    npmatrix = Tfidfmatrix.toarray()
    npmatrix[npmatrix == 0] = np.nan
    npmeans = np.nanmean(npmatrix, axis = 0)
    ordered_ids = np.argsort(npmeans)[::-1] 
    clean_ids = ordered_ids[(np.isnan(npmeans).sum()):]
    top_ids = clean_ids[:n]
    toptoken = [(tokens[i], npmeans[i]) for i in top_ids]
    df = pd.DataFrame(toptoken)
    df.columns = ['feature', 'TF-IDF']
    return df

def join_series(series_list):
    num_elements = series_list[0].shape[0]
    string_list = list()
    for i in range(num_elements):
        new_string = str()
        for column, value in enumerate(series_list):
            new_string = new_string + unidecode.unidecode(series_list[column][i]) + ' '
        string_list.append(new_string)
    new_series = pd.Series(data = string_list, index = range(num_elements))
    return new_series

In [3]:
class Display:
    
    def __init__(self):
        
        # Select file button
        self._bsf = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._bsf.add_traits(files=traitlets.traitlets.List())
        self._bsf.description = "Select File"
        self._bsf.icon = "square-o"
        self._bsf.on_click(self.select_excel)
        
        
        # Load Excel button
        self._blj = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._blj.description = "Load File"
        self._blj.icon = 'download'
        self._blj.on_click(self.load_excel)
        self._blj.disabled = True
                
        # Spreadsheet Dropdown
        style = {'description_width': 'initial'}
        self._dds = widgets.Dropdown(options=['Select file'], value='Select file', disabled=True, style=style,
                                     description='Spreadsheet:')
        
        # Column Selection
        self._csm = widgets.SelectMultiple(options=['Select table'], value =['Select table'], disabled=True)
        
        # Column Selection button
        self._bcs = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._bcs.description = "Choose Columns"
        self._bcs.on_click(self.choose_columns)
        self._bcs.disabled = True
        
        # define layout
        box_layout = widgets.Layout(display='center',
                            flex_flow='row',
                            align_items='center',
                            border='None',
                            justify_content='space-between',
                            width='900px')
        
        self._box = widgets.Box(children=[self._bsf, self._dds, self._blj,
                                          self._csm, self._bcs], layout=box_layout, main_size = 10)
        display(self._box)
        
    def select_excel(self,b): 
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        
        new_file = filedialog.askopenfilename(multiple=True,filetypes = [("XLS",'*.xls')])
        if new_file != "":
            self._bsf.files = new_file
            self._bsf.description = "File Selected"
            self._bsf.icon = "check-square-o"
            self._bsf.button_style = "success"
            self._blj.disabled = False
            print('Selected file: {}'.format(self._bsf.files))
            file_path = self._bsf.files
            self._excel = pd.ExcelFile(file_path[0], on_demand = True)
            self._sheets = self._excel.sheet_names
            self._dds.options = self._sheets
            self._dds.disabled = False

    def load_excel(self, b):
        self._dataframe = pd.read_excel(self._bsf.files[0], sheet_name = self._dds.value) 
        self._blj.button_style = "success"
        display(self._dataframe.head())
        self._csm.disabled = False
        self._csm.options = list(self._dataframe.columns.values)
        self._csm.disabled = False
        self._bcs.disabled = False
    
    def choose_columns(self, b):
        self._documents = list()
        for element in self._csm.value:
            self._dataframe[element].fillna("", inplace=True)
            self._documents.append( self._dataframe[element])
        self._documents = join_series(self._documents)
        print(self._documents)
        

In [4]:
d = Display()

Box(children=(Button(description='Select File', icon='square-o', layout=Layout(height='50px', width='500px'), …

In [5]:
counter = CountVectorizer(tokenizer=tokenization_process)
count_matrix = counter.fit_transform(d._documents)
terms = np.array(counter.get_feature_names())
index = np.array(range(len(terms)))

In [8]:
dicionario = pd.DataFrame.from_csv('Dicionario.csv')
dicionario.reset_index(inplace = True)
vec_dicionario = dicionario['Words'].values
for index, element in enumerate(vec_dicionario):
    vec_dicionario[index] = unidecode.unidecode(element.lower())
vec_dicionario = np.unique(vec_dicionario)

In [10]:
def word_checker(count_matrix, terms, repeats=100):
    #count_matrix em sparse
    #terms em np.array
    if len(terms) != count_matrix.shape[1]:
        raise ValueError("Terms and Count Matrix are not a match.")
    else:
        index_terms = np.array(range(len(terms)))
        token_frequency = count_matrix.sum(0)
        existent_words = terms[(token_frequency>repeats).A1]
        existent_words_index = index_terms[(token_frequency>repeats).A1]
        dubious_words = terms[(token_frequency<=repeats).A1]
        dubious_words_index = index_terms[(token_frequency<=repeats).A1]
        inexistent_words = np.array([])
        inexistent_words_index = np.array([])
        for number, term in enumerate(dubious_words):
            if term in vec_dicionario:
                existent_words = np.append(existent_words, term)
                existent_words_index = np.append(existent_words_index, dubious_words_index[number])
            else:
                inexistent_words = np.append(inexistent_words, term)
                inexistent_words_index = np.append(inexistent_words_index, dubious_words_index[number])
        return existent_words, inexistent_words, existent_words_index, inexistent_words_index

In [11]:
existent_words, inexistent_words, existent_words_index, inexistent_words_index =  word_checker(count_matrix, terms)

In [29]:
import time
begin = time.time()
edit_distance_matrix = [[edit_distance_nltk.edit_distance(i, j) for i in existent_words] for j in inexistent_words]
print(time.time()-begin)

1510.5069901943207


In [30]:
edit_distance_matrix = np.array(edit_distance_matrix)

In [54]:
word_switcheroo = list()
for index_old_word, inexistent_word_line in enumerate(edit_distance_matrix):
    if np.min(inexistent_word_line) < 3:
        
        if len(inexistent_word_line[inexistent_word_line == np.min(inexistent_word_line)]) > 1:
            max_freq_check = existent_words_index[inexistent_word_line == np.min(inexistent_word_line)]
            count_matrix_sums = [count_matrix[:,element_freq_check].sum() for element_freq_check in max_freq_check]
            index_new_word = max_freq_check[np.argmax(count_matrix_sums)]
            word_switcheroo.append([inexistent_words[index_old_word], terms[index_new_word]])
        else:
            word_switcheroo.append([inexistent_words[index_old_word], terms[existent_words_index[inexistent_word_line == np.min(inexistent_word_line)]][0]])

In [56]:
wrong_words = list()
right_words = list()
for mini_list in word_switcheroo:
    wrong_words.append(mini_list[0])
    right_words.append(mini_list[1])

In [58]:
def tokenization_process_removing_errors(string):
    """tokenizes one string, removing not used punctuation in portuguese language in the process"""
    aux = tokenizer(string)
    output = list()
    for index, word in enumerate(aux):
        if re.search('([A-Za-zÀ-ÿ]+(-|)[A-Za-zÀ-ÿ]+|[A-Za-zÀ-ÿ]+)', str(word)):
            if unidecode.unidecode(str(re.search('([A-Za-zÀ-ÿ]+(-|)[A-Za-zÀ-ÿ]+|[A-Za-zÀ-ÿ]+)', str(word)).group(0)).lower()) in wrong_words:
                output.append(right_words[wrong_words ==unidecode.unidecode(str(re.search('([A-Za-zÀ-ÿ]+(-|)[A-Za-zÀ-ÿ]+|[A-Za-zÀ-ÿ]+)', str(word)).group(0)).lower())])
            else:
                output.append(unidecode.unidecode(str(re.search('([A-Za-zÀ-ÿ]+(-|)[A-Za-zÀ-ÿ]+|[A-Za-zÀ-ÿ]+)', str(word)).group(0)).lower()))
    return output

In [40]:
count_matrix.shape

(8037, 10175)

In [32]:
edit_distance_matrix.shape

(5009, 5166)

In [62]:
import pandas as pd #dataframes e series
import matplotlib.pyplot as plt #graphics
import numpy as np #matrixes
import nltk #Tokenize
import sklearn as skl

from nltk.tokenize import word_tokenize
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer #tf-idf; tf
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [59]:
#INPUT
TFIDF = TfidfVectorizer(tokenizer = tokenization_process_removing_errors)
input_tfidf = TFIDF.fit_transform(d._documents)

In [60]:
#OUTPUT
output_wo_type = d._dataframe['Tipo Trabalho']
output_wo_values = output_wo_type.values
output_wo_values[output_wo_values =='GAR'] = 'MC'
output_wo_values[output_wo_values =='AN'] = 'MPR'
output_wo_values[output_wo_values =='ML'] = 'MPR'
output_wo_values[output_wo_values =='RP'] = 'MC'
output_wo_values[output_wo_values =='GPA'] = 'MPR'
output_wo_values[output_wo_values =='MPR'] = 1
output_wo_values[output_wo_values =='MC'] = 0
labels = pd.Series(data = output_wo_values, index = range(len(output_wo_values)),dtype = np.int32)

In [63]:
Kfolder = skl.model_selection.KFold(n_splits=10,shuffle = True, random_state= 100)

In [64]:
def featureselect(Model, X, Y):
    Model.fit(X, Y)
    print('Model Fited')
    featureselectmodel = SelectFromModel(Model, prefit = True)
    NewX = featureselectmodel.transform(X)
    return NewX

In [65]:
def cross_validation_score(Model,X,Y, cores = -1):
    return skl.model_selection.cross_val_score(Model, X, Y, cv = Kfolder, n_jobs = cores, scoring= 'f1').mean()

In [66]:
def Test(Model, X, Y):
    Score = list()
    for ind, column in enumerate(col_labels):
        Score.append(cross_validation_score(Model, X, Y[ind]))
        print('Score Created')
    print(sum(Score)/len(Score))
    return Score

In [67]:
#NBSVM Input
def pr(y_i, y, x):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)
def get_input_nbsvm(y, x):
    y = y.values
    r = np.log(pr(1,y,x) / pr(0,y,x))
    x_nb = x.multiply(r)
    return x_nb

In [ ]:
pre_processed_data = [input_tfidf,count_matrix,cbtw_matrix]
input_tfidf_scores = list()
count_matrix_scores = list()
cbtw_matrix_scores = list()
for iteration in range(0,11):
    for index, data in enumerate(pre_processed_data):
        intermediate_data = data
        Kfolder = skl.model_selection.KFold(n_splits=10,shuffle = True, random_state= iteration)
        model_input = get_input_nbsvm(labels, intermediate_data)
        score = cross_validation_score(SVM, model_input, labels)
        if index % 3 == 0:
            input_tfidf_scores.append(score)
        elif index % 3 == 1:
            count_matrix_scores.append(score)
        elif index % 3 == 2:
            cbtw_matrix_scores.append(score)

In [73]:
TF_IDF_treated = TfidfVectorizer(tokenizer = tokenization_process_removing_errors)
TF_IDF_untreated = TfidfVectorizer(tokenizer = tokenization_process)
input_tfidf_treated = TF_IDF_treated.fit_transform(d._documents)
input_tfidf_untreated = TF_IDF_untreated.fit_transform(d._documents)

In [76]:
model_input_treated = get_input_nbsvm(labels, input_tfidf_treated)
model_input_untreated = get_input_nbsvm(labels, input_tfidf_untreated)
score_list_treated = list()
score_list_untreated = list()
for iteration in range(0,11):
    Kfolder = skl.model_selection.KFold(n_splits=10,shuffle = True, random_state= iteration)
    score_list_treated.append(cross_validation_score(SVM, model_input_treated, labels))
    score_list_untreated.append(cross_validation_score(SVM, model_input_untreated, labels))
print(np.mean(score_list_treated))
print(np.mean(score_list_untreated))

0.8807025452397087
0.8923201764272857


In [68]:
nbsvm_input = get_input_nbsvm(labels, input_tfidf)

In [71]:
cross_validation_score(SVM, nbsvm_input, labels)

0.8808536309617079

In [ ]:
print(np.mean(input_tfidf_scores))
print(np.mean(count_matrix_scores))
print(np.mean(cbtw_matrix_scores))

In [ ]:
nbsvm_input = get_mdl(labels)

In [ ]:
cross_validation_score(SVM, nbsvm_input, labels)

In [ ]:
type(count_matrix)

In [70]:
TREE = DecisionTreeClassifier()
LOG = LogisticRegression()
SVM = skl.svm.LinearSVC()
NB = MultinomialNB()
MLP = MLPClassifier(hidden_layer_sizes=(5,))

In [ ]:
models = [TREE, LOG, SVM, NB, MLP]

In [80]:
cross_validation_score(TREE, cbtw_matrix, labels)

0.9241449178233834

In [ ]:
for i in range(2,11):
    print(cross_validation_score(MLPClassifier(hidden_layer_sizes=(i,),random_state=100), input_tfidf, labels))

In [77]:
import Class
import importlib
importlib.reload(Class)

<module 'Class' from 'C:\\Users\\Sérgio Jesus\\Documents\\Sergio2018\\Class.py'>

In [78]:
Document_Dataframe = Class.DocumentDataFrame(d._documents)

In [79]:
cbtw_matrix = Document_Dataframe.cbtw_matrix(labels)

<class 'scipy.sparse.csr.csr_matrix'>


In [ ]:
skl.svm.libsvm.cross_validation(input_tfidf, labels)

In [ ]:
np_list = np.array(new_list)

In [ ]:
np.save('comparison_list.npy', new_list)

In [ ]:
best_match = np.argmin(new_list, axis = 0)
for inexistent, existent in enumerate(best_match):
    Correction_id = np.array()
    print(inexistent)
    print(inexistent_words[inexistent])
    print(existent_words[existent])

In [ ]:
id_switch = np.array([[inexistent_words_index[inexistent].astype(int),existent_words_index[existent].astype(int)]
                       for inexistent, existent in enumerate(best_match)])

In [ ]:
id_switch

In [ ]:
def tf_idf_error_remover(documents):
    timevariable = time.time()
    counter = CountVectorizer(tokenizer=tokenization_process)
    print(time.time()-timevariable)
    timevariable = time.time()
    count_matrix = counter.fit_transform(documents)
    print(time.time()-timevariable)
    timevariable = time.time()
    terms = np.array(counter.get_feature_names())
    print(time.time()-timevariable)
    timevariable = time.time()
    vec_dictionary = create_dictionary_nparray('Dicionario.csv')
    print(time.time()-timevariable)
    timevariable = time.time()
    existent_words, inexistent_words, existent_words_index, inexistent_words_index =  word_checker(count_matrix, terms)
    print(time.time()-timevariable)
    timevariable = time.time()
    distance_matrix = edit_distance_matrix(inexistent_words, existent_words) #bottleneck
    print(time.time()-timevariable)
    timevariable = time.time()
    id_switch = id_switcher_matrix(distance_matrix, existent_words_index, inexistent_words_index)


In [ ]:
def edit_distance_matrix(first_term_vector, second_term_vector): #bottleneck
    return np.array([[edit_distance_nltk.edit_distance(i, j) for i in first_term_vector] for j in second_term_vector])

In [ ]:
def id_switcher_matrix(distance_matrix, inexistent_words_index, existent_words_index):
    best_match = np.argmin(distance_matrix, axis = 0)
    id_switch = np.array([[inexistent_words_index[inexistent].astype(int),existent_words_index[existent].astype(int)]
                       for inexistent, existent in enumerate(best_match)])
    return id_switch

In [ ]:
def create_dictionary_nparray(path):
    dictionary = pd.DataFrame.from_csv(path)
    dictionary.reset_index(inplace = True)
    vec_dictionary = dicionario['Words'].values
    for index, element in enumerate(vec_dictionary):
        vec_dictionary[index] = unidecode.unidecode(element.lower())
    return vec_dictionary

In [72]:
word_switcheroo

[['aaeronave', 'aeronave'],
 ['aaf', 'aa'],
 ['aalrme', 'alarme'],
 ['aarme', 'alarme'],
 ['aauto-level', 'auto-level'],
 ['abanado', 'abana'],
 ['abandonamos', 'abandonado'],
 ['abandonei', 'abandonar'],
 ['abanou', 'acabou'],
 ['abatimentos', 'abatimento'],
 ['abolsa', 'bolsa'],
 ['about', 'aut'],
 ['above', 'abre'],
 ['abracadeiras', 'bracadeira'],
 ['abrandou', 'arrancou'],
 ['abri-la', 'abril'],
 ['abria', 'abrir'],
 ['abriam', 'abrir'],
 ['acabando', 'acabado'],
 ['acabarem', 'acabar'],
 ['acabava', 'acabado'],
 ['acabe', 'acaba'],
 ['acaitacao', 'aceitacao'],
 ['acamponhada', 'acompanhada'],
 ['accedez', 'aceder'],
 ['access', 'acesso'],
 ['accionando', 'accionado'],
 ['acederam', 'aceder'],
 ['acedi', 'acende'],
 ['acedido', 'cedido'],
 ['aceesso', 'acesso'],
 ['aceiracao', 'aceitacao'],
 ['aceit', 'aceite'],
 ['aceitaao', 'aceitacao'],
 ['aceitacaode', 'aceitacao'],
 ['aceitacap', 'aceitacao'],
 ['aceitaco', 'aceitacao'],
 ['aceitacoes', 'aceitamos'],
 ['aceitada', 'aceitado']